# Genesis4 Basic Example

This will show the basic usage of LUME-Genesis.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import genesis
from genesis.version4 import Genesis4, Track, Write

%config InlineBackend.figure_format = 'retina'

## Setup

Instantiate the object on a value init file. This will configure a working directory that stages all input and output files.

In [ ]:
G = Genesis4("data/basic4/cu_hxr.in", "data/basic4/hxr.lat", verbose=True)

Here is what the main input ("cu_hxr.in") looks like in Python:

In [ ]:
G.input.main

Inspect the main input by filtering the namelists by their type:

In [ ]:
G.input.main.tracks

There is also a dictionary mapping you can use to see all types by using their type class directly:

In [ ]:
G.input.main.by_namelist[Track]

We can use this to modify the z-stop for all track instances:

In [ ]:
for track in G.input.main.tracks:
    track.zstop = 12

In [ ]:
G.input.main.tracks

You can view the data as a convenient table, too, when looking at a single namelist:

In [ ]:
G.input.main.tracks[0].to_table()

In [ ]:
# Add writing a field file
G.input.main.namelists.append(Write(field="end"))

# Add writing a beam (particle) file
G.input.main.namelists.append(Write(beam="end"))

Let's look at the lattice input format that Genesis will see:

In [ ]:
print(G.input.lattice.to_genesis())

Here's what the Python representation would look like:

In [ ]:
G.input.lattice

Here's what a fully-detailed table representation looks like:

In [ ]:
G.input.lattice.to_table()

You can turn off/on descriptions in tables and configure other aspects of LUME-Genesis rendering of objects for Jupyter:

In [ ]:
genesis.global_display_options.include_description = False
G.input.main.writes[0].to_table()

View it as a markdown table for easy pasting into GitHub and other platforms:

In [ ]:
print(G.input.main.profile_files[0].to_string("markdown"))

## Run
Run with MPI. Here, setting `G.nproc = 0` will automatically select the maximum number of CPUs.

In [ ]:
G.nproc = 0
output = G.run()

In [ ]:
if output.run.error:
    print(G.output.run.error_reason)
else:
    print("No error")

## Output

The main output is an HDF5. The Genesis4 object loads all array data into convenient Pydantic models:

In [ ]:
G.output.beam

In [ ]:
G.output.beam.stat.alphax.shape

In [ ]:
plt.plot(G.output.beam.stat.sigma_x)
plt.yscale("log")

Each of these parameters has a string alias that you can use to refer to the data.  These string parameters can be used in `G.plot()` and similar methods.

Here are the first 10 of those keys:

In [ ]:
list(G.output.keys())[:10]

In [ ]:
print("Took", G.output.run.run_time, "sec")

To inspect these aliases, check this dictionary:

In [ ]:
print(G.output.alias["alphax"])

In [ ]:
G.output["alphax"] is G.output.beam.alphax

The above indicates that the alias `"alphax"` corresponds to the attribute ``beam.alphax``. So, you could access the data either as `G.output["alphax"]` or `G.output.beam.alphax`.

Though the former may be shorter, you can take advantage of tab completion when working with the Python classes directly.  Try typing `G.output.beam.` and then hit tab with the cursor after the final `.`. You should see a list of other output beam data.

There are many outputs. `.output_info()` gives a convenient table describing what was read in.

In [ ]:
genesis.global_display_options.include_description = True

In [ ]:
G.output.info()

## Fields

Field files can be very large and are made readily available for lazy loading.
Loaded fields are present in `.field` attribute of the output:

In [ ]:
list(G.output.field3d)

For convenience, fields and particles may be automatically loaded after a run by using `run(load_fields=True, load_particles=True)` instead.
Otherwise, these can be manually loaded individually or all at once:

In [ ]:
G.output.load_fields()

In [ ]:
list(G.output.field3d)

This field data has two parts: basic parameters `param`, and the raw 3D complex array `dfl`:

In [ ]:
G.output.field3d["end"].param

In [ ]:
G.output.field3d["end"].dfl.shape

`.field` is a convenience property that points to this

In [ ]:
G.output.field3d["end"].param

## Visualize Field

In [ ]:
# Sum over y and compute the absolute square
dfl = G.output.field3d["end"].dfl
param = G.output.field3d["end"].param
dat2 = np.abs(np.sum(dfl, axis=1)) ** 2

In [ ]:
plt.imshow(dat2)

In [ ]:
def plot_slice(i=0):
    dat = np.angle(dfl[:, :, i])

    dx = param.gridsize * 1e6
    plt.xlabel("x (µm)")
    plt.xlabel("y (µm)")
    plt.title(f"Phase for slize {i}")
    plt.imshow(dat.T, origin="lower", extent=[-dx, dx, -dx, dx])


plot_slice(i=100)

# Particles

Particle files can be read in as [openPMD-beamphysics](https://christophermayes.github.io/openPMD-beamphysics/) `ParticleGroup` objects.
These are loaded only on-demand by default (`run(load_particles=False)`). They may also be loaded all at once, with `load_particles()`.

In [ ]:
list(G.output.particle_files)

In [ ]:
G.output.load_particles()

After loading particles, use the `.particles` attribute to access their data:

In [ ]:
G.output.particles

In [ ]:
P = G.output.particles["end"]

In [ ]:
P.plot("z", "energy")

Change to z coordinates to see the current. Note that the head of the bunch is now on the left.

In [ ]:
P.drift_to_z()
P.plot("t", "energy")

Check some statistics

In [ ]:
P["norm_emit_x"], P["norm_emit_y"], P["mean_gamma"]

## Bunching

In [ ]:
wavelength = G.input.main.setup.lambda0
bunching_key = f"bunching_{wavelength}"
P.drift_to_t()

P.slice_plot(bunching_key, n_slice=1000)

In [ ]:
# Genesis4 data
final_bunching = G.output.beam.bunching[-1, :]
current = G.output.beam.current[-1, :]
s = G.output.globals.s


# ParticleGroup data
ss = P.slice_statistics(bunching_key, n_slice=len(s))
ss.keys()
x = ss["mean_z"]
y = ss[bunching_key]

Compare 

In [ ]:
fig, ax = plt.subplots()
ax.plot(x * 1e6, y, label="ParticleGroup")
ax.plot(s * 1e6, final_bunching, "--", label="Genesis4 output")
ax.set_xlabel("s (µm)")
ax.set_ylabel("bunching")
plt.legend()

This is the average bunching from the ParticleGroup:

In [ ]:
P.bunching(wavelength)

That agrees with the appropriate averaging of Genesis4's bunching calc:

In [ ]:
G.stat("bunching")[-1]

In [ ]:
G.plot("bunching")

Check the total charge in pC:

In [ ]:
P["charge"] / 1e-12  # pC

## Units

Each item in the output class should have corresponding units.

In [ ]:
G.output.units("beam_betax")

# Plotting

Convenient plotting of the data in `.output` is provided by `.plot`. The default is to plot the power. Depending on the key these statistics are averaged or integrated over the slices. Some keys like `power` are converted to `peak_power`, while `field_energy` is the integral over `field_power`. 

In [ ]:
print(G.output.alias["power"])

In [ ]:
G.plot()

Left and right axes can be set this way:

In [ ]:
G.plot("field_energy", yscale="log", y2=["beam_xsize", "beam_ysize"], ylim2=(0, 100e-6))

By default, these plots average over slices. In the case of beam sizes, simply averaging these does not take into account the effect of misaligned slices. To plot this, LUME-Genesis provides additional `beam_sigma_x`, `beam_sima_y`, `beam_sigma_energy` keys that properly project these quantities. The difference is noticable in the energy spread calculation:

In [ ]:
G.plot(["beam_sigma_energy", "beam_energyspread"], ylim=(0, 100))

In [ ]:
G.plot(["field_xsize", "field_ysize"])

In [ ]:
plt.imshow(G.output.field.power, aspect="auto");

## Archiving

In [ ]:
G.archive("archived.h5")

Grestored = Genesis4.from_archive("archived.h5")

In [ ]:
Grestored.output.plot()

### Check that the restored object is the same

The Genesis4 object as well as the input, output, and all namelists support equality checks:

In [ ]:
Grestored == G

In [ ]:
Grestored.input == G.input

In [ ]:
Grestored.output == G.output

# Manual loading of Genesis4 data

Sometimes it is necessary to run Genesis4 manually, and load the output into LUME-Genesis for further analysis.

First, let's create some input to run in a local directory `temp/`:

In [ ]:
import os

os.makedirs("temp/", exist_ok=True)
G.write_input("temp/")

Now run on the command line:

In [ ]:
!cd temp; ./run | tee log.txt

Using the `use_temp_dir=False` and `workdir` options, the input and output data can be loaded into a new Genesis4 object:

In [ ]:
G2 = Genesis4("genesis4.in", use_temp_dir=False, workdir="temp/", verbose=True)
G2.configure()
output = G2.load_output()

In [ ]:
output.plot()

# Cleanup

In [ ]:
import shutil

shutil.rmtree("temp")
os.remove("archived.h5")